In [2]:
%%sh
cat > lpi-mpi.c << EOF
#include <stdio.h>
#include <stdlib.h>
#include <mpi.h>

#define N 100
#define ilosc_procesow 1

int main(int argc, char *argv[]) {
    int rank, size;
    MPI_Init(&argc, &argv);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    int start = rank * (N / ilosc_procesow) + 1;
    int end = start + (N / ilosc_procesow) - 1;
    int i;
    double sum = 0, avg;
    for (i = start; i <= end; i++) {
        sum += i;
    }
    avg = sum / (end - start + 1);
    printf("Proces %d wylicza sumę od %d do %d = %f i średnią = %f \n", rank, start, end, sum, avg);

    if (rank != ilosc_procesow - 1) {
        MPI_Send(&sum, 1, MPI_DOUBLE, rank + 1, 0, MPI_COMM_WORLD);
    } else {
        double total_sum = sum;
        for (i = 0; i < ilosc_procesow - 1; i++) {
            MPI_Recv(&sum, 1, MPI_DOUBLE, i, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
            total_sum += sum;
        }
        double total_avg = total_sum / N;
        printf("Globalna suma wszystkich liczb: %f \n", total_sum);
        printf("Globalna średnia wszystkich liczb: %f \n", total_avg);
    }

    MPI_Finalize();
    return 0;
}
EOF
mpicc lpi-mpi.c && mpirun -n 1 --allow-run-as-root a.out

Proces 0 wylicza sumę od 1 do 100 = 5050.000000 i średnią = 50.500000 
Globalna suma wszystkich liczb: 5050.000000 
Globalna średnia wszystkich liczb: 50.500000 
